In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json, time, os, sys, glob
import shutil
import warnings
import numpy as np
import pandas as pd
import torch
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split, Subset
import copy
import torch.nn as nn
import torch.nn.functional as F
import random
import os.path
import subprocess
from tqdm import tqdm
from omegaconf import OmegaConf
import pytorch_lightning as pl
from terrace.batch import DataLoader
from typing import Sequence

from fireprot_dataset import *
from protein_mpnn_utils import loss_nll, loss_smoothed, gather_edges, gather_nodes, gather_nodes_t, cat_neighbors_nodes, _scores, _S_to_seq, tied_featurize, parse_PDB
from protein_mpnn_utils import StructureDataset, StructureDatasetPDB, ProteinMPNN
from kaggle_dataset import KaggleTrainDataset
from transfer_model import *
from train import *

In [3]:
cfg = OmegaConf.load("config.yaml")

In [106]:
from collections import defaultdict

def msa_hist(msa_file):
    counts = None
    with open(msa_file, "r") as f:
        for line in f.readlines():
            if line.startswith(">"): continue
            if counts is None:
                counts = [ defaultdict(int) for c in line ]
            for i, c in enumerate(line.upper()):
                if i > len(counts) - 1:
                    break
                counts[i][c] += 1
    alphabet = 'ACDEFGHIKLMNPQRSTVWY-'
    ret = torch.zeros((len(counts), len(alphabet)), dtype=torch.float32)
    for i, count_dict in enumerate(counts):
        for j, c in enumerate(alphabet):
            ret[i][j] = count_dict[c]
    return ret/ret.sum(-1).unsqueeze(-1)

msa_hist("data/msas/P00883.a3m")[-1]

tensor([0.0305, 0.0047, 0.0068, 0.0070, 0.0053, 0.0125, 0.0123, 0.0030, 0.0214,
        0.0059, 0.0015, 0.0131, 0.0038, 0.0072, 0.0047, 0.0167, 0.0127, 0.0201,
        0.0011, 0.0858, 0.7241])

In [125]:
dataset = FireProtDataset(cfg, "val")
for _ in tqdm(dataset):
    pass

No msa for P13487 (1), skipping
No msa for Q7SIG1 (1), skipping
No msa for P29957 (1), skipping
No msa for P24297 (1), skipping
No msa for P61991 (1), skipping
No msa for P11053 (1), skipping
No msa for P00173 (1), skipping
No msa for P22307 (1), skipping


KeyboardInterrupt: 

In [63]:
rows = df.query("sequence == 'MVRPTILLTSLLLAPFAAASPILEERQAAQSVDQLIKARGKVYFGVATDQNRLTTGKNAAIIQADFGQVTPENSMKWDATEPSQGNFNFAGADYLVNWAQQNGKLIRGHTLVWHSQLPSWVSSITDKNTLTNVMKNHITTLMTRYKGKIRAWDVVNEAFNEDGSLRQTVFLNVIGEDYIPIAFQTARAADPNAKLYINDYNLDSASYPKTQAIVNRVKQWRAAGVPIDGIGSQTHLSAGQGAGVLQALPLLASAGTPEVAITELDVAGASPTDYVNVVNACLNVQSCVGITVWGVADPDSWRASTTPLLFDGNFNPKPAYNAIVQDLQQ'").reset_index(drop=True)
pdb_seq = "AAAQSVDQLIDARGKVYFGVATDQNRLTTGKNAAIIQADFGQVTPENSMKWDATEPSQGNFNFAGADYLVNWAQQNGKLIRGHTLVWHSQLPSWVVSITDKNTLTNVMKNHITTIMTRYIGKIRAWDVVNEAFNEDGSLRQTVFNNVIGEDYIPIAFRTARAADPNAKLYINDYNLDSASKPKTSAIVKRVKKWRAAGVPIDGIGSQTHLSAGQGASIDAALPNLASAGTPEVAITELDIAGATSTDYVDVVNACLDVDSCIGITVWGVADPDSWRASTTPLLFDGNFNPKPAYNAIVQLL"
rows.position[0], rows.position_corrected[0], rows.sequence[0][rows.position[0]], rows.sequence[0][rows.position_corrected[0]], pdb_seq[rows.position[0]], pdb_seq[rows.position_corrected[0]], rows.wild_type[0]

(235, 208, 'L', 'K', 'T', 'H', 'H')

In [33]:
for _ in dataset:
    pass

In [49]:
model = TransferModel()
model(pdb, mut)[0]["ddG"]

tensor([-0.1385], grad_fn=<UnsqueezeBackward0>)

In [51]:
model_pl = TranserModelPL(cfg)
model_pl.shared_eval((pdb, mut), 0, "val")

/home/boris/miniconda3/envs/chem/lib/python3.8/site-packages/pytorch_lightning/core/lightning.py:415: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


tensor([2.2188], grad_fn=<AddBackward0>)

In [9]:
import pandas as pd
len(pd.read_csv("data/v2_dataset_12072022/fireprotDB_cleaned_allproteins_allmutations.csv").uniprot_id.unique())

218

In [82]:
dataset = FireProtDataset(cfg, "train")
# train_data = [ [ d ] for d in dataset ]

In [83]:
model_pl = TranserModelPL(cfg)
trainer = pl.Trainer()
trainer.fit(model_pl, dataset, dataset)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name    | Type          | Params
------------------------------------------
0 | model   | TransferModel | 1.7 M 
1 | metrics | ModuleDict    | 0     
------------------------------------------
10.8 K    Trainable params
1.7 M     Non-trainable params
1.7 M     Total params
6.685     Total estimated model params size (MB)


TypeError: Expected data to be int, Sequence or Mapping, but got FireProtDataset